In [ ]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

# 下载停用词（如果还没有下载过）
nltk.download('stopwords')

# 定义数据预处理函数
def preprocess_text(text):
    # 小写化
    text = text.lower()
    # 去除特殊字符和数字，保留汉字和英文字符
    text = re.sub(r'[^a-zA-Z\u4e00-\u9fa5]', ' ', text)
    return text

# 读取数据集
df = pd.read_excel('tieba.xlsx')  # 替换为你的文件名

# 假设 '标题' 列是你的文本，'分类（杂谈1、互助2、生活分享3、交流合作4、闲置5）' 列是你的标注数据
texts = df['标题'].tolist()
labels = df['分类（杂谈1、互助\t2、生活分享3、交流合作4、闲置5）'].tolist()

# 数据预处理
processed_texts = [preprocess_text(text) for text in texts]

# 将标签转换为整数并减去 1
labeled_labels = [int(label) - 1 for label in labels if pd.notna(label)]
labeled_texts = [text for text, label in zip(processed_texts, labels) if pd.notna(label)]

# 确保文本和标签的长度一致
assert len(labeled_texts) == len(labeled_labels), "文本和标签长度不一致"

# 将手动标注的数据划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    labeled_texts, labeled_labels, test_size=0.2, random_state=42, stratify=labeled_labels)

# 加载BERT分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# 对文本进行编码
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128)

# 创建数据集类
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 创建训练集和测试集
train_dataset = TextDataset(train_encodings, y_train)
test_dataset = TextDataset(test_encodings, y_test)

# 加载BERT模型
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=5)

# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results2', #定义模型存放位置
    num_train_epochs=4, 
    #设置训练的 epoch 数，即重复学习次数，初始为3，当调整为10时，在第五次后出现了过拟合现象，所以采用：4次
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500, #定义学习率预热的步骤数
    weight_decay=0.01, #权重衰减系数（也称为 L2 正则化），用于防止过拟合
    logging_dir='./logs',
    logging_steps=10,
    # evaluation_strategy="epoch" # 未来版本可能不再支持
    eval_strategy="epoch",  
)

# 使用Trainer API进行训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 训练模型
trainer.train()

# 预测测试集的分类
predictions, labels, _ = trainer.predict(test_dataset)
predicted_labels = predictions.argmax(-1)

# 打印分类报告
print("分类报告：")
print(classification_report(y_test, predicted_labels))

# 将预测的标签和原始文本保存到 DataFrame 中
output_df = pd.DataFrame({
    '标题': X_test,
    '真实标签': y_test,
    '预测标签': predicted_labels
})

# 将结果保存到 Excel 文件
output_df.to_excel('prediction_results.xlsx', index=False)

print("预测结果已保存到 'prediction_results.xlsx'")

# 对未标注的数据进行预测并保存

# 提取未标注的数据
unlabeled_texts = df[df['分类（杂谈1、互助\t2、生活分享3、交流合作4、闲置5）'].isna()]['标题'].tolist()

# 打印未标注的数据数量和内容，确保数据被正确提取
print(f"未标注的数据数量: {len(unlabeled_texts)}")
# print(f"未标注的文本数据: {unlabeled_texts}")


# 如果有未标注的数据，进行处理
if len(unlabeled_texts) > 0:
    # 对未标注的文本进行编码
    unlabeled_encodings = tokenizer(unlabeled_texts, truncation=True, padding=True, max_length=128)

    # 创建未标注数据集
    class UnlabeledTextDataset(torch.utils.data.Dataset):
        def __init__(self, encodings):
            self.encodings = encodings

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            return item

        def __len__(self):
            return len(self.encodings['input_ids'])

    # 创建未标注数据集
    unlabeled_dataset = UnlabeledTextDataset(unlabeled_encodings)

    # 使用训练好的模型对未标注数据进行预测
    predictions, _, _ = trainer.predict(unlabeled_dataset)
    predicted_labels = predictions.argmax(-1)

    # 将预测的标签和未标注文本保存到 DataFrame 中
    unlabeled_df = pd.DataFrame({
        '标题': unlabeled_texts,
        '预测标签': predicted_labels
    })

    # 将未标注数据的预测结果填充到原始 DataFrame 中对应的位置
    df.loc[df['分类（杂谈1、互助\t2、生活分享3、交流合作4、闲置5）'].isna(), '分类（杂谈1、互助\t2、生活分享3、交流合作4、闲置5）'] = predicted_labels + 1

    # 将包含预测结果的原始数据保存到 Excel 文件
    df.to_excel('prediction_results_with_unlabeled.xlsx', index=False)

    print("未标注数据的预测结果已保存到 '重复5次的预测数据.xlsx'")
else:
    print("没有未标注的数据。")
